## dense matrix

In [1]:
import numpy as np

x = np.array([[3, 0, 0], 
              [0, 2, 0],
              [1, 0, 1],
              [2, 0, 1]
             ])
x

array([[3, 0, 0],
       [0, 2, 0],
       [1, 0, 1],
       [2, 0, 1]])

In [2]:
# marginal probability
px = x.sum(axis=1) / x.sum()
py = x.sum(axis=0) / x.sum()

print(px)
print(py)

[0.3 0.2 0.2 0.3]
[0.6 0.2 0.2]


In [3]:
# convert x to probability matrix
pxy = x / x.sum()
print(pxy)

[[0.3 0.  0. ]
 [0.  0.2 0. ]
 [0.1 0.  0.1]
 [0.2 0.  0.1]]


In [4]:
# diagonalize px & py for matrix multiplication
# (4, 4) by (4, 3) by (3, 3) = (4, 3)
np.diag(px)

array([[0.3, 0. , 0. , 0. ],
       [0. , 0.2, 0. , 0. ],
       [0. , 0. , 0.2, 0. ],
       [0. , 0. , 0. , 0.3]])

In [5]:
# inverse elements if the element is greater than 0
np.diag(np.array([0 if pxi == 0 else 1/pxi for pxi in px]))

array([[3.33333333, 0.        , 0.        , 0.        ],
       [0.        , 5.        , 0.        , 0.        ],
       [0.        , 0.        , 5.        , 0.        ],
       [0.        , 0.        , 0.        , 3.33333333]])

In [6]:
# inverse element diagonal matrix of px and py
px_diag = np.diag(np.array([0 if pxi == 0 else 1/pxi for pxi in px]))
py_diag = np.diag(np.array([0 if pyi == 0 else 1/pyi for pyi in py]))

print(px_diag.shape)
print(py_diag.shape)

(4, 4)
(3, 3)


In [7]:
# logarithm is not applied yet
exp_pmi = px_diag.dot(pxy).dot(py_diag)

exp_pmi

array([[1.66666667, 0.        , 0.        ],
       [0.        , 5.        , 0.        ],
       [0.83333333, 0.        , 2.5       ],
       [1.11111111, 0.        , 1.66666667]])

In [8]:
# check value
# difference cause by truncation error
for i in range(x.shape[0]):
    for j in range(x.shape[1]):
        exp_pmi_ij = exp_pmi[i,j]
        manually_ij = pxy[i,j] / (px[i] * py[j])
        if not (exp_pmi_ij == manually_ij):
            print('({}, {}), exp_pmi = {}, manually = {}'.format(
                i, j, exp_pmi_ij, manually_ij))

(1, 1), exp_pmi = 5.0, manually = 4.999999999999999
(2, 2), exp_pmi = 2.5, manually = 2.4999999999999996
(3, 0), exp_pmi = 1.1111111111111114, manually = 1.1111111111111112
(3, 2), exp_pmi = 1.666666666666667, manually = 1.6666666666666667


In [9]:
# PPMI  = max(0, PMI)
# Select (i, j) pairs having greater pmi value
#   and store the value into dok_matrix for fast retrieval

from scipy.sparse import dok_matrix

# PPMI using threshold
min_exp_pmi = 1

rows, cols = np.where(exp_pmi > min_exp_pmi)
pmi_dok = dok_matrix(exp_pmi.shape)

for i, j in zip(rows, cols):
    # apply logarithm
    pmi_dok[i,j] = np.log(exp_pmi[i,j])

for position, pmi_ij in pmi_dok.items():
    print('{} = {} (exp = {})'.format(
        position, pmi_ij, np.exp(pmi_ij)))

(0, 0) = 0.5108256237659907 (exp = 1.6666666666666667)
(1, 1) = 1.6094379124341003 (exp = 4.999999999999999)
(2, 2) = 0.9162907318741551 (exp = 2.5)
(3, 0) = 0.10536051565782655 (exp = 1.1111111111111114)
(3, 2) = 0.5108256237659908 (exp = 1.666666666666667)


## sparse matrix

In [10]:
from scipy.io import mmread

x = mmread('/mnt/lovit/git/retrieval_bot/tmp/mc50_word_context.mtx').tocsr()
with open('/mnt/lovit/git/retrieval_bot/tmp/mc50_vectorizer.vocab', encoding='utf-8') as f:
    idx2vocab = [word.strip() for word in f]

vocab2idx = {vocab:idx for idx, vocab in enumerate(idx2vocab)}
print(vocab2idx['뭐먹'], vocab2idx['어디'], vocab2idx['피자'], vocab2idx['치킨'], vocab2idx['지하철'])

17170 17729 15043 16990 17161


In [11]:
x.shape

(17761, 17761)

In [12]:
# convert x to probability matrix & marginal probability 
px = (x.sum(axis=1) / x.sum()).reshape(-1)
py = (x.sum(axis=0) / x.sum()).reshape(-1)
pxy = x / x.sum()

print(px.shape, py.shape, pxy.shape)

(1, 17761) (1, 17761) (17761, 17761)


In [13]:
# transform px and py to diagonal matrix
# using scipy.sparse.diags

from scipy.sparse import diags

px_diag = diags(px.tolist()[0])
py_diag = diags((py).tolist()[0])

In [14]:
px_diag.shape

(17761, 17761)

In [15]:
print(type(px_diag.data))
print(px_diag.data.shape)

<class 'numpy.ndarray'>
(1, 17761)


In [16]:
# inverse element diagonal matrix of px and py with alpha smoothing

alpha = 0.0001 # acts as p(y) threshold

px_diag.data[0] = np.asarray([0 if v == 0 else 1/v for v in px_diag.data[0]])
py_diag.data[0] = np.asarray([0 if v == 0 else 1/(v + alpha) for v in py_diag.data[0]])

In [17]:
type(px_diag), type(pxy), type(py_diag), 

(scipy.sparse.dia.dia_matrix,
 scipy.sparse.csr.csr_matrix,
 scipy.sparse.dia.dia_matrix)

In [18]:
exp_pmi = px_diag.dot(pxy).dot(py_diag)

exp_pmi.shape

(17761, 17761)

In [19]:
# PPMI using threshold
min_exp_pmi = 1

# because exp_pmi is sparse matrix and type of exp_pmi.data is numpy.ndarray
indices = np.where(exp_pmi.data > min_exp_pmi)[0]

In [36]:
pmi_dok = dok_matrix(exp_pmi.shape)

# prepare data (rows, cols, data)
rows, cols = exp_pmi.nonzero()
data = exp_pmi.data

# enumerate function for printing status
for _n_idx, idx in enumerate(indices):
    
    # print current status
    if _n_idx % 10000 == 0:
        print('\r{:.3} %'.format(100 * _n_idx / indices.shape[0]), flush=True, end='')
        
    # apply logarithm
    pmi_dok[rows[idx], cols[idx]] = np.log(data[idx])

1e+02 %

In [20]:
for term_1 in ['뭐먹', '어디']:
    for term_2 in ['피자', '치킨', '지하철']:
        term_1_idx = vocab2idx[term_1]
        term_2_idx = vocab2idx[term_2]
        pmi_12 = pmi_dok[term_1_idx, term_2_idx]
        print('PPMI({}, {}) = {}'.format(term_1, term_2, pmi_12))

IndexError: index out of bounds

In [21]:
submatrix = pmi_dok[term_2_idx,:].tocsr()
contexts = submatrix.nonzero()[1]
values = submatrix.data
most_relateds = [(idx, value) for idx, value in zip(contexts, values)]
most_relateds = sorted(most_relateds, key=lambda x:-x[1])[:topk]

IndexError: index (15043) out of range -4 to 3)

## using soynlp

In [22]:
import sys
sys.path.append('/mnt/lovit/git/soynlp/')
from soynlp.word import pmi
pmi_soynlp = pmi(x, verbose=True)

computing pmi was done                              


In [23]:
for term_1 in ['뭐먹', '어디']:
    for term_2 in ['피자', '치킨', '지하철']:
        term_1_idx = vocab2idx[term_1]
        term_2_idx = vocab2idx[term_2]
        pmi_12 = pmi_soynlp[term_1_idx, term_2_idx]
        print('PPMI({}, {}) = {}'.format(term_1, term_2, pmi_12))

PPMI(뭐먹, 피자) = 3.0861874233694397
PPMI(뭐먹, 치킨) = 3.8521326246077767
PPMI(뭐먹, 지하철) = 0.0
PPMI(어디, 피자) = 0.0
PPMI(어디, 치킨) = 0.4396905736037917
PPMI(어디, 지하철) = 2.2598710194560514


In [ ]:
from soynlp